In [113]:
import pandas as pd
import numpy as np
import copy
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

df = pd.read_excel("Asurion_data.xlsx")
additional_df = pd.read_excel("apple_additional_data_clean.xlsx")

## Run initial data through cleaning steps

### Fix issue with storage size

In [114]:
import re
def convert_size(args):
    model, size = args
    if size == 'gb':
        num = re.search(r'\d+$', model).group()
        model_re = re.sub(r" ?\d+$", "", model)
        return pd.Series([model_re, ''.join([num, 'gb'])])
    else:
        return pd.Series([model, size])

df[['phone model', 'phone size']] = df[['phone model', 'phone size']].apply(convert_size, axis = 1)

In [118]:
df['phone size'].unique()

array(['32gb', '4gb', '16gb', '128gb', '256gb', '64gb', '512gb', '8gb'],
      dtype=object)

### Version Merging

In [119]:
df['phone model'] = df['phone model'].str.replace('mini', '')
df['phone model'] = df['phone model'].str.replace('max', '')
df['phone model'] = df['phone model'].str.replace('xl', '')

df['phone model'] = df['phone model'].str.replace(' plus ', ' + ')
df['phone model'] = df['phone model'].str.replace('plus', '')
df['phone model'] = df['phone model'].str.replace('+', 'plus')

# Kyocera Durforce Pro
df['phone model'] = df['phone model'].str.replace(' pro ', ' - ')
df['phone model'] = df['phone model'].str.replace('pro', '')
df['phone model'] = df['phone model'].str.replace('-', 'pro')
df['phone model'] = df['phone model'].str.replace('2020', '')
df['phone model'] = df['phone model'].str.replace('2022', '')

df['phone model'] = df['phone model'].str.replace('3g s', '3gs')
df['phone model'] = df['phone model'].str.replace('5g', '')
# df['phone model'] = df['phone model'].str.replace('4a', '4')
df['phone model'] = df['phone model'].str.replace('5c', '5s')

df['phone model'] = df['phone model'].str.replace(' 32', '')
df['phone model'] = df['phone model'].str.replace(' 64', '')
df['phone model'] = df['phone model'].str.replace(' 128', '')
df['phone model'] = df['phone model'].str.replace(' 256', '')
df['phone model'] = df['phone model'].str.replace('ultra', '')
df['phone model'] = df['phone model'].str.replace('active', '')
df['phone model'] = df['phone model'].str.replace('edge', '')
df['phone model'] = df['phone model'].str.replace('10e', '10')

df['phone model'] = df['phone model'].str.strip()
df['phone model'].unique()

/var/folders/qc/xhd08p257qs0hgklv6c67x640000gn/T/ipykernel_79637/4185803233.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['phone model'] = df['phone model'].str.replace('+', 'plus')


array(['alcatel axel', 'alcatel smartflip 4052r', 'alcatel tetra',
       'apple iphone 11', 'apple iphone 12', 'apple iphone 13',
       'apple iphone 14', 'apple iphone 3gs', 'apple iphone 4',
       'apple iphone 4s', 'apple iphone 5', 'apple iphone 5s',
       'apple iphone 6', 'apple iphone 6s', 'apple iphone 7',
       'apple iphone 8', 'apple iphone se', 'apple iphone x',
       'apple iphone xr', 'apple iphone xs', 'at t cingular flip iv',
       'google pixel 4', 'google pixel 4a', 'google pixel 5',
       'google pixel 6', 'kyocera duraforce pro 2', 'lg arena 2',
       'lg g6 duo', 'lg g8 thinq', 'lg g8x thinq', 'lg k30', 'lg k40',
       'lg k92', 'lg neon', 'lg phoenix 5', 'lg prime 2', 'lg stylo 4',
       'lg stylo 5', 'lg v40 thinq', 'lg v60 thinq', 'lg velvet',
       'lg wing', 'lg xpression plus 2', 'lg xpression plus 3',
       'motorola one  ace', 'nokia 3.1 a', 'red hydrogen one',
       'samsung galaxy a01', 'samsung galaxy a02s', 'samsung galaxy a10',
       'sa

### Feature Engineering

In [120]:
df["brand"] = df["phone model"].str.split(" ", n = 1, expand = True)[0]
df["model"] = df["phone model"].str.split(" ", n = 1, expand = True)[1]
df["generation"] = df["model"].str.split(" ", n = 1, expand = True)[1]
df["model"] = df["model"].str.split(" ", n = 1, expand = True)[0]
df["week_of_month"] = df["weeks_monday"].dt.day.apply(lambda x: (x-1)//7+1)
df["month"] = df["weeks_monday"].dt.month
df["year"] = df["weeks_monday"].dt.year

df

,phone model,phone size,phone color,claim,weeks_monday,brand,model,generation,week_of_month,month,year
0,alcatel axel,32gb,black,1,2022-01-24,alcatel,axel,None,4,1,2022
1,alcatel axel,32gb,black,3,2022-11-14,alcatel,axel,None,2,11,2022
2,alcatel axel,32gb,black,1,2022-11-21,alcatel,axel,None,3,11,2022
3,alcatel axel,32gb,black,3,2022-11-28,alcatel,axel,None,4,11,2022
4,alcatel axel,32gb,black,1,2022-12-12,alcatel,axel,None,2,12,2022
...,...,...,...,...,...,...,...,...,...,...,...
26656,samsung galaxy z fold3,256gb,silver,17,2023-01-30,samsung,galaxy,z fold3,5,1,2023
26657,samsung galaxy z fold3,256gb,silver,26,2023-02-06,samsung,galaxy,z fold3,1,2,2023
26658,samsung galaxy z fold3,256gb,silver,4,2023-02-13,samsung,galaxy,z fold3,2,2,2023
26659,samsung galaxy z fold3,512gb,black,61,2023-02-06,samsung,galaxy,z fold3,1,2,2023


### Add "weeks_since_release" feature

In [121]:
df['release'] = pd.to_datetime(np.nan)

#iphone 3gs released on June 19, 2009
df.loc[df['phone model'] == 'apple iphone 3gs', 'release'] = '2009-06-19'

#iphone 4 released on 2010-06-24
df.loc[(df['brand'] == 'apple') & (df['generation'] == '4'), 'release'] = '2010-06-24'

#for phone_model iphone 4s, set the release date to 2011-10-14
df.loc[(df['brand'] == 'apple') & (df['generation'] == '4s'), 'release'] = '2011-10-14'

#for phone_model iphone 5, set the release date to 2012-09-21
df.loc[df['phone model'] == 'apple iphone 5', 'release'] = '2012-09-21'

#for phone model iphone 5s, set the release date to 2013-09-20
df.loc[df['phone model'] == 'apple iphone 5s', 'release'] = '2013-09-10'

#iphone 5s and 5c released on 2013-09-20
df.loc[df['phone model'] == 'apple iphone 5c', 'release'] = '2013-09-10'

#iphone 6 and 6 plus released on 2014-09-19
df.loc[(df['brand'] == 'apple') & (df['generation'] == '6'), 'release'] = '2014-09-19'

#iphone 6 and 6 plus released on 2014-09-19
df.loc[(df['brand'] == 'apple') & (df['generation'] == '6 plus'), 'release'] = '2014-09-19'

#for phone model iphone 6s, set the release date to 2015-09-15
df.loc[(df['brand'] == 'apple') & (df['generation'] == '6s'), 'release'] = '2015-09-15'

#iphone se march 31,2016
df.loc[(df['brand'] == 'apple') & (df['generation'] == 'se'), 'release'] = '2016-03-31'

#for brand apple and generation 7, in the column release, set the release date to 2016-09-16
df.loc[(df['brand'] == 'apple') & (df['generation'] == '7'), 'release'] = '2016-09-16'

#for brand apple and generation 8, in the column release, set the release date to 2017-09-22
df.loc[(df['brand'] == 'apple') & (df['generation'] == '8'), 'release'] = '2017-09-22'

#for brand apple and generation x, in the column release, set the release date to 2017-11-03
df.loc[(df['brand'] == 'apple') & (df['generation'] == 'x'), 'release'] = '2017-11-03'

#for brand apple and generation xr, in the column release, set the release date to 2018-10-26
df.loc[(df['brand'] == 'apple') & (df['generation'] == 'xr'), 'release'] = '2018-10-26'

#for brand apple and generation xs, in the column release, set the release date to 2018-09-21
df.loc[(df['brand'] == 'apple') & (df['generation'] == 'xs'), 'release'] = '2018-09-21'

#for brand apple and generation 11, in the column release, add the value 2019-09-20
df.loc[(df['brand'] == 'apple') & (df['generation'] == '11'), 'release'] = '2019-09-20'

#for brand apple and generation 12, in the column release, add the value 2020-11-13
df.loc[(df['brand'] == 'apple') & (df['generation'] == '12'), 'release'] = '2020-11-13'

#for brand apple and generation 13, in the column release, add the value 2021-09-24
df.loc[(df['brand'] == 'apple') & (df['generation'] == '13'), 'release'] = '2021-09-24'

#for brand apple and generation 14, in the column release, add the value 2022-09-16
df.loc[(df['brand'] == 'apple') & (df['generation'] == '14'), 'release'] = '2022-09-16'
    
#for brand apple and generation 14 plus, in the column release, add the value 2022-10-06
df.loc[(df['brand'] == 'apple') & (df['generation'] == '14 plus'), 'release'] = '2022-10-06'

df['weeks_since_release'] = df['weeks_monday'] - df['release']
df['weeks_since_release'] = df['weeks_since_release'].dt.days / 7

apple = df[df['brand'] == 'apple']
apple

,phone model,phone size,phone color,claim,weeks_monday,brand,model,generation,week_of_month,month,year,release,weeks_since_release
32,apple iphone 11,128gb,black,111,2021-06-28,apple,iphone,11,4,6,2021,2019-09-20,92.428571
33,apple iphone 11,128gb,black,319,2021-07-05,apple,iphone,11,1,7,2021,2019-09-20,93.428571
34,apple iphone 11,128gb,black,320,2021-07-12,apple,iphone,11,2,7,2021,2019-09-20,94.428571
35,apple iphone 11,128gb,black,305,2021-07-19,apple,iphone,11,3,7,2021,2019-09-20,95.428571
36,apple iphone 11,128gb,black,269,2021-07-26,apple,iphone,11,4,7,2021,2019-09-20,96.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17906,apple iphone xs,64gb,silver,18,2023-01-16,apple,iphone,xs,3,1,2023,2018-09-21,225.428571
17907,apple iphone xs,64gb,silver,19,2023-01-23,apple,iphone,xs,4,1,2023,2018-09-21,226.428571
17908,apple iphone xs,64gb,silver,14,2023-01-30,apple,iphone,xs,5,1,2023,2018-09-21,227.428571
17909,apple iphone xs,64gb,silver,13,2023-02-06,apple,iphone,xs,1,2,2023,2018-09-21,228.428571


In [122]:
apple['release'].isnull().sum()

0

In [123]:
apple[apple['release'].isnull()]['generation'].unique()

array([], dtype=object)

In [124]:
apple['phone model'].unique()

array(['apple iphone 11', 'apple iphone 12', 'apple iphone 13',
       'apple iphone 14', 'apple iphone 3gs', 'apple iphone 4',
       'apple iphone 4s', 'apple iphone 5', 'apple iphone 5s',
       'apple iphone 6', 'apple iphone 6s', 'apple iphone 7',
       'apple iphone 8', 'apple iphone se', 'apple iphone x',
       'apple iphone xr', 'apple iphone xs'], dtype=object)

### Add notion of holidays

In [125]:
# create a dictionary with the holiday seasons
holidays = {
    'Christmas': ['12-25', '12-31'],
    'Black Friday': ['11-28', '11-29'],
    'Back to School': ['06-21', '09-27'],
    'Summer Vacation': ['06-01', '08-31']
}

In [126]:
# label the holiday seasons
for holiday, date_range in holidays.items():
    start_date = f"{apple['weeks_monday'].iloc[0].year}-{date_range[0]}"
    end_date = f"{apple['weeks_monday'].iloc[0].year}-{date_range[1]}"
    apple.loc[(apple['weeks_monday'] >= start_date) & (apple['weeks_monday'] <= end_date), 'holiday_season'] = holiday

# fill the NAN with 0s and the other columns with 1s
apple['is_holiday'] = apple['holiday_season'].apply(lambda x: 0 if pd.isna(x) else 1)
apple = apple.drop('holiday_season', axis=1)

# print the resulting dataframe
apple

/var/folders/qc/xhd08p257qs0hgklv6c67x640000gn/T/ipykernel_79637/3421680808.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apple.loc[(apple['weeks_monday'] >= start_date) & (apple['weeks_monday'] <= end_date), 'holiday_season'] = holiday
/var/folders/qc/xhd08p257qs0hgklv6c67x640000gn/T/ipykernel_79637/3421680808.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apple['is_holiday'] = apple['holiday_season'].apply(lambda x: 0 if pd.isna(x) else 1)


,phone model,phone size,phone color,claim,weeks_monday,brand,model,generation,week_of_month,month,year,release,weeks_since_release,is_holiday
32,apple iphone 11,128gb,black,111,2021-06-28,apple,iphone,11,4,6,2021,2019-09-20,92.428571,1
33,apple iphone 11,128gb,black,319,2021-07-05,apple,iphone,11,1,7,2021,2019-09-20,93.428571,1
34,apple iphone 11,128gb,black,320,2021-07-12,apple,iphone,11,2,7,2021,2019-09-20,94.428571,1
35,apple iphone 11,128gb,black,305,2021-07-19,apple,iphone,11,3,7,2021,2019-09-20,95.428571,1
36,apple iphone 11,128gb,black,269,2021-07-26,apple,iphone,11,4,7,2021,2019-09-20,96.428571,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17906,apple iphone xs,64gb,silver,18,2023-01-16,apple,iphone,xs,3,1,2023,2018-09-21,225.428571,0
17907,apple iphone xs,64gb,silver,19,2023-01-23,apple,iphone,xs,4,1,2023,2018-09-21,226.428571,0
17908,apple iphone xs,64gb,silver,14,2023-01-30,apple,iphone,xs,5,1,2023,2018-09-21,227.428571,0
17909,apple iphone xs,64gb,silver,13,2023-02-06,apple,iphone,xs,1,2,2023,2018-09-21,228.428571,0


(Source: https://www.bankmycell.com/blog/iphone-evolution-timeline-chart#text)

## Combine Datasets

In [127]:
apple.columns, additional_df.columns

(Index(['phone model', 'phone size', 'phone color', 'claim', 'weeks_monday',
        'brand', 'model', 'generation', 'week_of_month', 'month', 'year',
        'release', 'weeks_since_release', 'is_holiday'],
       dtype='object'),
 Index(['phone model', 'phone size', 'phone color', 'weeks_monday', 'claim',
        'brand', 'model', 'generation', 'week_of_month', 'month', 'year',
        'release', 'weeks_since_release', 'is_holiday'],
       dtype='object'))

In [128]:
claim = additional_df.pop('claim')
additional_df.insert(3, 'claim', claim)

In [129]:
apple.columns, additional_df.columns

(Index(['phone model', 'phone size', 'phone color', 'claim', 'weeks_monday',
        'brand', 'model', 'generation', 'week_of_month', 'month', 'year',
        'release', 'weeks_since_release', 'is_holiday'],
       dtype='object'),
 Index(['phone model', 'phone size', 'phone color', 'claim', 'weeks_monday',
        'brand', 'model', 'generation', 'week_of_month', 'month', 'year',
        'release', 'weeks_since_release', 'is_holiday'],
       dtype='object'))

## Remove datapoints that are present in additional data

In [130]:
apple['day'] = pd.to_datetime(apple['weeks_monday']).dt.day

apple = apple[apple['weeks_monday'] != '2023-02-06']
apple = apple[apple['weeks_monday'] != '2023-02-13']

## Combine datasets

In [131]:
df_combined = apple.append(additional_df)
print(len(apple), len(additional_df), len(df_combined))

17458 1026 18484


/var/folders/qc/xhd08p257qs0hgklv6c67x640000gn/T/ipykernel_79637/3048057137.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = apple.append(additional_df)


In [132]:
df_combined['phone size'] = df_combined['phone size'].apply(lambda x: int(x.replace('gb', '')))

# Drop unneeded columns - identical over all rows, or correlated with other engineered columns
df_combined.drop(['model', 'brand','release'], inplace=True, axis=1)

In [134]:
df_combined.columns, df_combined['phone size'].unique()

(Index(['phone model', 'phone size', 'phone color', 'claim', 'weeks_monday',
        'generation', 'week_of_month', 'month', 'year', 'weeks_since_release',
        'is_holiday', 'day'],
       dtype='object'),
 array([128, 256,  64, 512,   8,  16,  32]))

In [139]:
df_combined.to_excel('apple_full_data_clean.xlsx', index=False)

In [136]:
test_time = [
    '2023-02-27',
    '2023-02-20',
    '2023-02-13',
    '2023-02-06',
    '2023-01-30',
    '2023-01-23',
    '2023-01-16',
    '2023-01-09',
    '2023-01-02',
    '2022-12-26',
    '2022-12-19'
]

test_data = df_combined.loc[df_combined['weeks_monday'].isin(test_time)]
train_data = df_combined.loc[~df_combined['weeks_monday'].isin(test_time)]

In [138]:
# Sanity check that we only have the dates we wanted.
print("Dates in Test Data:", test_data['weeks_monday'].unique())
print("Size of Train Data:", len(train_data), "; Size of Test Data:", len(test_data))
print("Train Data is ", len(train_data)/(len(test_data) + len(train_data)) * 100, "% of Full Data")
print("Test Data is ", len(test_data)/(len(test_data) + len(train_data)) * 100, "% of Full Data")

Dates in Test Data: ['2022-12-19T00:00:00.000000000' '2022-12-26T00:00:00.000000000'
 '2023-01-02T00:00:00.000000000' '2023-01-09T00:00:00.000000000'
 '2023-01-16T00:00:00.000000000' '2023-01-23T00:00:00.000000000'
 '2023-01-30T00:00:00.000000000' '2023-02-06T00:00:00.000000000'
 '2023-02-13T00:00:00.000000000' '2023-02-20T00:00:00.000000000'
 '2023-02-27T00:00:00.000000000']
Size of Train Data: 15873 ; Size of Test Data: 2611
Train Data is  85.87426963860636 % of Full Data
Test Data is  14.125730361393638 % of Full Data


In [140]:
train_data.reset_index(drop=True)
train_data.to_excel("apple_train_data_final.xlsx", index=False)

test_data.reset_index(drop=True)
test_data.to_excel("apple_test_data_final.xlsx", index=False)